In [1]:
import json

dic_points = {}
  
# Opening JSON file
f = open(r"C:\Users\jorge\Desktop\all_via_region_data.json")

# returns JSON object as 
# a dictionary
data = json.load(f)

dic_points = {}
ids_images = []

# Iterating through the json
# list
for key, value in data.items():
    regions = value["regions"]
    filename = value["filename"]
    file = filename[-26:]
    for it in regions:
        it = it["shape_attributes"]
        px = it["all_points_x"]
        py = it["all_points_y"]
        poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
        # poly = [p for x in poly for p in x]
        # poly = np.array(poly)

        dic_points[file] = [poly]
        ids_images.append(file[4:22])

# Closing file
f.close()

len(dic_points)

29995

In [2]:
import cv2
import numpy as np
import csv, os, shutil

path_source = r'C:/Users/jorge/Desktop/ALL_IMAGES_JSON/'
path_dest = r'C:/Users/jorge/Desktop/ALL_IMAGES_JSON_MASKS/'

with open(r'C:\Users\jorge\Desktop\GalaxyZoo1_DR_table2.csv', newline='') as File:
    reader = csv.reader(File)
    for row in reader:
        if row[0] in ids_images:
            key = 'Dr7_' + row[0] + '.jpg'
            file_source = path_source + key
            if os.path.isfile(file_source):
                im = cv2.imread(file_source)
                pts = np.array(dic_points[key], np.int32)
                mask = np.zeros((im.shape[0], im.shape[1], im.shape[2]), np.float64)
                
                color = (0, 0, 0)
                if int(row[13]) == 1:
                    color = (128, 128, 128)
                elif int(row[14]) == 1:
                    color = (100, 100, 100)
                elif int(row[15]) == 1:
                    color = (194, 194, 194)
                
                for i in range(im.shape[0]):
                    for j in range(im.shape[1]):
                        if cv2.pointPolygonTest(pts, [j,i], False) != -1.0:
                            mask[i,j,:] = color
                
                file_dest = path_dest + 'Dr7_' + row[0] + '.png'
                cv2.imwrite(file_dest, mask)